In [1]:
import nibabel as nib
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import skimage

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
bunny = o3d.data.BunnyMesh()
gt_mesh = o3d.io.read_triangle_mesh(bunny.path)
gt_mesh.compute_vertex_normals()

pcd = gt_mesh.sample_points_poisson_disk(3000)
print(pcd)
print(pcd.points)
# o3d.visualization.draw_geometries([pcd])

PointCloud with 3000 points.
std::vector<Eigen::Vector3d> with 3000 elements.
Use numpy.asarray() to access data.


In [3]:
radii = [0.005, 0.01, 0.02, 0.04]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii))
# o3d.visualization.draw_geometries([pcd, rec_mesh])

In [4]:
my_pcd = o3d.io.read_point_cloud("constructions/sub-001.ply")

In [5]:
o3d.visualization.draw_geometries([my_pcd])

In [6]:
# Compute the axis-aligned bounding box of the point cloud
bounding_box = my_pcd.get_axis_aligned_bounding_box()

# Get the minimum and maximum corners of the bounding box
min_bound = bounding_box.get_min_bound()
max_bound = bounding_box.get_max_bound()

# Compute the scale factors needed for normalization
scale_factors = max_bound - min_bound

# Avoid division by zero in case of degenerate (flat) point clouds
scale_factors[scale_factors == 0] = 1

# Translate and scale the points
normalized_points = (np.asarray(my_pcd.points) - min_bound) / scale_factors

# Create a new point cloud for the normalized points
normalized_pcd = o3d.geometry.PointCloud()
normalized_pcd.points = o3d.utility.Vector3dVector(normalized_points)

In [7]:
my_pcd.normals = o3d.utility.Vector3dVector(np.zeros(
    (1, 3))) 
my_pcd.estimate_normals()
o3d.visualization.draw_geometries([my_pcd], point_show_normal=True)

In [8]:
my_pcd.orient_normals_consistent_tangent_plane(100)
o3d.visualization.draw_geometries([my_pcd], point_show_normal=True)

In [34]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        my_pcd, depth=11)
print(mesh)
o3d.visualization.draw_geometries([mesh])

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 79070 / 79070
[Open3D DEBUG] #   Got kernel density: 0.49095916748046875 (s), 739.98046875 (MB) / 739.98046875 (MB) / 746 (MB)
[Open3D DEBUG] #     Got normal field: 0.42777299880981445 (s), 829.73046875 (MB) / 829.73046875 (MB) / 829 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 4.689845e-05 / 3.708261e+00
[Open3D DEBUG] #       Finalized tree: 1.0830230712890625 (s), 929.98046875 (MB) / 929.98046875 (MB) / 929 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.8193440437316895 (s), 835.6328125 (MB) / 929.98046875 (MB) / 929 (MB)
[Open3D DEBUG] #Set point constraints: 0.20771002769470215 (s), 835.7578125 (MB) / 929.98046875 (MB) / 929 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 4537604 / 1214808 / 3971025
[Open3D DEBUG] Memory Usage: 835.758 MB
Cycle[0] Depth[ 0/11]:	Updated constraints / Got system / Solved in:  0.000 /  0.000 /  0.000	(835.758 MB)	Nodes: 8
CG: 1.3130e+00 -> 1.3130e+00 -> 1.015

In [64]:
o3d.visualization.draw_geometries([mesh])

In [65]:
o3d.io.write_triangle_mesh("constructions/sub-001_mesh.obj", mesh)

True

## Point cloud ball poviting

In [35]:
distances = my_pcd.compute_nearest_neighbor_distance()
average_distance = np.mean(distances)

In [36]:
average_distance

1.000057624246694

In [59]:
radii = [average_distance * 2, average_distance * 2.1, average_distance * 8]
my_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    my_pcd, o3d.utility.DoubleVector(radii))
print(my_mesh)
o3d.visualization.draw_geometries([my_mesh])

TriangleMesh with 79070 points and 81675 triangles.
